In [1]:
#general definitions and imports
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import os
import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging( logging_level='DEBUG')

import numpy as np
import matplotlib.pyplot as plt

import unittest

from pathlib import Path

from PySpice.Unit import *
from PySpice.Spice.Parser import SpiceParser
from PySpice.Spice.Netlist import SubCircuitFactory
from PySpice.Spice.Library import SpiceLibrary
from PySpice.Probe.Plot import plot
from PySpice.Math import *

## setup spice library path
directory_path = Path(os.path.abspath('')).resolve().parent.parent
spice_libraries_path = directory_path.joinpath("lib", "spice")
spice_library = SpiceLibrary(spice_libraries_path)

directory_path = Path(os.path.abspath('')).resolve()


2020-11-18 21:30:00,684 - PySpice.Spice.Netlist.__new__ - DEBUG - Element don't define a __pins__ attribute
2020-11-18 21:30:00,685 - PySpice.Spice.Netlist.__new__ - DEBUG - FixedPinElement don't define a __pins__ attribute
2020-11-18 21:30:00,687 - PySpice.Spice.Netlist.__new__ - DEBUG - SubCircuitElement don't define a __pins__ attribute
2020-11-18 21:30:00,689 - PySpice.Spice.Netlist.__new__ - DEBUG - Resistor don't define a __pins__ attribute
2020-11-18 21:30:00,691 - PySpice.Spice.Netlist.__new__ - DEBUG - SemiconductorResistor don't define a __pins__ attribute
2020-11-18 21:30:00,695 - PySpice.Spice.Netlist.__new__ - DEBUG - BehavioralResistor don't define a __pins__ attribute
2020-11-18 21:30:00,696 - PySpice.Spice.Netlist.__new__ - DEBUG - Capacitor don't define a __pins__ attribute
2020-11-18 21:30:00,699 - PySpice.Spice.Netlist.__new__ - DEBUG - SemiconductorCapacitor don't define a __pins__ attribute
2020-11-18 21:30:00,701 - PySpice.Spice.Netlist.__new__ - DEBUG - Behaviora

![module](https://img.shields.io/badge/module-mischer-yellow)
![status](https://img.shields.io/badge/status-work%20in%20progress-orange)

# *oberwelle*

<a href="https://photos.app.goo.gl/pg6iZUC32rTKX5LC7"><img src="https://spielhuus.github.io/elektrophon/images/kontrast-logo-tmb.jpg" height="300px" align="right"></a>

***table of contents***

* [*about*](#about)
* [*construction*](#construction)
* [*calibration*](#calibration)
* [*usage*](#usage)
* [*references*](#lreferences)
* [*changelog*](#changelog)

<br/><br/>

## *about*



## construction

because the channel is silent at potentometer center position, it would be nice to use a potentometer that has a center decent. such potientometers are not easy to find. some examples are [[10][10]]


<img src="main/main.svg"/>


In [2]:
#load the Overdrive 250 schema
kicad_netlist_path = directory_path.joinpath('middle/multiplier.cir')
parser = SpiceParser(path=str(kicad_netlist_path))

class VoltageDivider1(SubCircuitFactory):
    __name__ = 'voltage_divider_1'
    __nodes__ = ('n1', 'n2', 'n3' )
    def __init__(self, R=100@u_kΩ, w=0.4):
        super().__init__()
        self.R(1, 'n1', 'n2', R * w)
        self.R(2, 'n2', 'n3', R * (1.0-w) )

class VoltageDivider2(SubCircuitFactory):
    __name__ = 'voltage_divider_2'
    __nodes__ = ('n1', 'n2', 'n3' )
    def __init__(self, R=100@u_kΩ, w=0.4):
        super().__init__()
        self.R(1, 'n1', 'n2', R * w)
        self.R(2, 'n2', 'n3', R * (1.0-w) )

class VoltageDivider3(SubCircuitFactory):
    __name__ = 'voltage_divider_3'
    __nodes__ = ('n1', 'n2', 'n3' )
    def __init__(self, R=100@u_kΩ, w=0.4):
        super().__init__()
        self.R(1, 'n1', 'n2', R * w)
        self.R(2, 'n2', 'n3', R * (1.0-w) )

def middlePlot( cv_atten, cv_level, input_level ) :
    middle = parser.build_circuit(ground=5)
    middle.include(spice_library['TL072c'])
    middle.include(spice_library['D1N4148'])
    middle.include(spice_library['CA3080'])
    middle.include(spice_library['BC556B'])
    middle.V('1', '+15V', middle.gnd, 'DC 15')
    middle.V('2', '-15V', middle.gnd, 'DC -15')
    middle.V('3', 'IN_1', middle.gnd, 'DC 0V AC 5V SIN(0 1V 500)')
    middle.V('4', 'IN_2', middle.gnd, 'DC 0V AC 0V SIN(0 0V 500)')
    middle.V('5', 'CV', middle.gnd, 'DC 0V AC 0V SIN(0 0V 500)')
    for subcircuit in (VoltageDivider1(R=100@u_kΩ, w=cv_atten), 
                               VoltageDivider2(R=100@u_kΩ, w=input_level),
                               VoltageDivider3(R=50@u_kΩ, w=cv_level) ) :
        middle.subcircuit(subcircuit)
    simulator = middle.simulator(temperature=25, nominal_temperature=25)
    return simulator.transient(step_time=1@u_us, start_time=100@u_ms, end_time=104@u_ms)

#plot the results
fig, axis = plt.subplots(figsize=(18, 6))
analysis_middle = middlePlot( 0.9999, 0.00001, 0.2 )    
#axis.plot(analysis_middle['IN_1'].abscissa*1000, analysis_middle['IN_1'], c='grey')  # envelope input (scaled)
#axis.plot(analysis_middle['IN_2'].abscissa*1000, analysis_middle['IN_2'], c='grey')  # envelope input (scaled)
#axis.plot(analysis_middle['Net-_R5-Pad2_'].abscissa*1000, analysis_middle['Net-_R5-Pad2_'], c='Red')  # envelope input (scaled)
#axis.plot(analysis_middle['Net-_R4-Pad1_'].abscissa*1000, analysis_middle['Net-_R4-Pad1_'], c='grey')  #CA3080 inverting input
#axis.plot(analysis_middle['Net-_C1-Pad1_'].abscissa*1000, analysis_middle['Net-_C1-Pad1_'], c='Brown')  #CA3080 non inverting input
#axis.plot(analysis_middle['Net-_Q1-Pad1_'].abscissa*1000, analysis_middle['Net-_Q1-Pad1_'], c='Pink')  # envelope input (scaled)
#axis.plot(analysis_middle['Net-_D1-Pad1_'].abscissa*1000, analysis_middle['Net-_D1-Pad1_'], c='LightBlue')  # envelope input (scaled)
axis.plot(analysis_middle['Net-_C2-Pad2_'].abscissa*1000, analysis_middle['Net-_C2-Pad2_'], c='LightGreen')  #CA3080 Output
#axis.plot(analysis_middle['OUT'].abscissa*1000, analysis_middle['OUT'], c='orange')  # envelope input (scaled)

axis.grid()
axis.set_xlabel('t [ms]')
axis.set_ylabel('[V]')
axis.set_title('middle section', y=-0.2)
plt.show()
 

2020-11-18 21:30:01,665 - PySpice.Spice.Parser.Element.build - DEBUG - X U1 ['Net-_C2-Pad1_', 'Net-_C2-Pad2_', 'GND', '-15V', 'Net-_D2-Pad2_', 'Net-_C3-Pad2_', 'Net-_C3-Pad1_', '+15V'] ['TL072c'] {}
2020-11-18 21:30:01,667 - PySpice.Spice.Parser.Element.build - DEBUG - X U2 ['Net-_C4-Pad1_', 'Net-_C4-Pad2_', 'Net-_D3-Pad2_', '-15V', 'Net-_D7-Pad2_', 'Net-_C6-Pad2_', 'Net-_C6-Pad1_', '+15V'] ['TL072c'] {}
2020-11-18 21:30:01,668 - PySpice.Spice.Parser.Element.build - DEBUG - R 9 ['Net-_C2-Pad1_', 'Net-_C2-Pad2_'] ['82k'] {}
2020-11-18 21:30:01,672 - PySpice.Spice.Parser.Element.build - DEBUG - R 14 ['Net-_C3-Pad1_', 'Net-_C3-Pad2_'] ['100k'] {}
2020-11-18 21:30:01,675 - PySpice.Spice.Parser.Element.build - DEBUG - D 4 ['Net-_D2-Pad2_', 'GND'] [] {'model': 'D1N4148'}
2020-11-18 21:30:01,678 - PySpice.Spice.Parser.Element.build - DEBUG - D 2 ['GND', 'Net-_D2-Pad2_'] [] {'model': 'D1N4148'}
2020-11-18 21:30:01,680 - PySpice.Spice.Parser.Element.build - DEBUG - R 10 ['Net-_C3-Pad2_', 'Net-_

NgSpiceCommandError: Command 'run' failed

## *calibration*

there is no calibration needed. but the potentiomenter knobs have to be aliged to center position.

## *usage*


## *references*

- [Op Amp Differential Amplifier][1] eCircuit  Center
- [Beginners' Guide to Potentiometers][2]  Rod Elliott (ESP)

[1]: http://www.ecircuitcenter.com/Circuits/opdif/opdif.htm
[2]: https://sound-au.com/pots.htm
[10]: https://www.mouser.ch/ProductDetail/ALPS/RK09L114001T?qs=Xb8IjHhkxj78e5CYccfZWg%3D%3D


## *changelog*



---
[![CC BY-SA](https://licensebuttons.net/l/by-sa/3.0/88x31.png)](https://creativecommons.org/licenses/by-sa/4.0/)